In [1]:
# Final EE vs ramp time (T_cut) for J in {2,1,0.5} — styled like previous figs

import numpy as np
import qutip as qt
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from pathlib import Path
from datetime import datetime
import matplotlib.patheffects as pe
from matplotlib.gridspec import GridSpec

# ====== Matplotlib: presentation style (big labels, inward ticks) ======
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('default')
mpl.rcParams.update({
    "axes.labelsize": 18,   # 軸ラベル
    "xtick.labelsize": 16,  # 目盛り数字
    "ytick.labelsize": 16,
    "lines.linewidth": 2.2, # 線の太さ
    "axes.titlesize": 16,
})

# ---- helpers (same vibe as earlier figs) ----
def label_right_of_last(ax, x, y, text, *, dx_pt=8, fontsize=16):
    """
    曲線の最後の点 (x[-1], y[-1]) の『右側』に、同じ高さでラベルを置く。
    - dx_pt: 右方向オフセット（ポイント）
    """
    ax.annotate(
        text,
        xy=(x[-1], y[-1]),
        xytext=(dx_pt, 0), textcoords="offset points",
        ha="left", va="center",
        fontsize=fontsize,
        bbox=dict(boxstyle="round,pad=0.25", facecolor="white", alpha=0.7, lw=0),
        path_effects=[pe.withStroke(linewidth=3, foreground="white")],
        clip_on=False,  # 右端に少しはみ出しても切れないように
        zorder=5,
    )
    
def inward_ticks(ax, major_len=7, minor_len=4, width=1.2):
    ax.minorticks_on()
    ax.tick_params(which="both", direction="in", top=True, right=True,
                   length=major_len, width=width)
    ax.tick_params(which="minor", length=minor_len, width=width)

def _screen_angle(ax, x, y, i):
    trans = ax.transData.transform
    i0, i1 = max(i-1, 0), min(i+1, len(x)-1)
    (x0, y0) = trans((x[i0], y[i0])); (x1, y1) = trans((x[i1], y[i1]))
    return np.degrees(np.arctan2(y1-y0, x1-x0))

# ---- 直書きラベル（凡例ボックスなし・重なり回避つき） ----
import matplotlib.patheffects as pe
import numpy as np

# ==== 追加: 手置きラベル用ヘルパ（凡例ボックスなし） ====
import matplotlib.patheffects as pe

def label_at(ax, x, y, text, *, coord='data', dx_pt=8, fontsize=14, color=None):
    """
    (x,y) にラベルを直接配置。
      coord='data'  -> データ座標
      coord='axes'  -> 軸比率座標 (0..1, 左下=0,0)
    dx_pt: 右方向の微小オフセット[pt]（線と重なりを避ける）
    """
    kw = dict(ha='left', va='center', fontsize=fontsize,
              xytext=(dx_pt, 0), textcoords='offset points',
              path_effects=[pe.withStroke(linewidth=3, foreground='white')],
              clip_on=False, zorder=6)
    if color is not None:
        kw['color'] = color
    if coord == 'axes':
        ax.annotate(text, xy=(x, y), xycoords=ax.transAxes, **kw)
    else:
        ax.annotate(text, xy=(x, y), xycoords='data', **kw)

# # ====== Save all ====== 
# TS = datetime.now().strftime("%Y%m%d-%H%M%S") 
# OUTDIR = Path("charts") / TS 
# OUTDIR.mkdir(parents=True, exist_ok=True) 
# def save_fig(fig, stem: str):
#     fig.savefig(OUTDIR / f"{stem}.pdf", bbox_inches="tight", dpi=300) 
#     print(f"saved: {OUTDIR / (stem + '.pdf')}") 